# **TP3 | Le chiffrement de Rabin**

Dans ce TP, on se concentre sur la procédure de chiffrement de Rabin, une des premières méthode de chiffrement à clefs asymétriques/clés publiques inventée en 1979. 
Nous présentons ici une version du chiffrement de Rabin modifié.

Voici le déroulé de cette dernière:
- Bob (le destinataire) génère deux nombres premiers $p$ et $q$ et communique $n = p q$ à Alice dans le canal public.
- Alice souhaite envoyé le message $m$ à Bob. Elle envoie $c=m^2 \mod n$ à Bob.
- Bob déchiffre $c$ en calculant ses racines carrés.

*Remarque :* 
Dans $\mathbb{Z} / n\mathbb{Z}$ avec $n$ non premier, la racine carrée n'est pas unique : ici, à chaque message il y a quatres racines potentielles.
C'est le problème majeur de cette méthode.
On dit que procédure de chiffrement est **non-injective**. 
Il faut donc avoir recours à des tests supplémentaires pour s'assurer que le message a été décrypté correctement, ce qui peut être difficile à mettre en place en pratique.

Les conversions entre $\mathbb{Z} / n\mathbb{Z}$ et l'espace produit $\mathbb{Z} / p\mathbb{Z} \times \mathbb{Z} / q\mathbb{Z}$ sont le sujets des deux premières parties.
La première se concentre sur l'algorithme d'Euclide étendu et le calcul des coefficients dits «de Bézout» qui
permettent de calculer des inverses et de résoudres des équations modulaires. 
La deuxième établit cette correspondance à proprement parler avec des fonctions de conversion.

En partie 3, on implémente le calcul de racine carrée dans $\mathbb{Z} / p\mathbb{Z}$ et le chiffrement de Rabin sur les nombres, sans la correspondance nombre - caractère (on ne pourra donc pas distinguer la «bonne» racine).

Toute cette procédure repose sur le fait que la décomposition en facteurs premiers de $n = p q$ est difficile à retrouver.
Le chiffrement RSA repose sur le même principe. 
C'est pourquoi la partie 4 étudie la complexité de cette factorisation.

In [ ]:
# Les imports utiles
import math
try:
    import galois
except:
    import os
    os.system("python -m pip install galois")
    import galois

import random
import time

DEFAULT_SEED = sum(ord(c) for c in "R3.09 - Cryptographie")

## Partie 1 - Algorithme d'Euclide étendu

On cherche des coefficients dits de Bézout $u,v$ tels que
$$ ua + vb = {\rm PGCD}(a, b). $$
Pour cela, on adopte une démarche récursive basée sur la division euclidienne, $a = qb + r$.

- Si $r = 0$, alors on a $u = 0$ et $v = 1$.
- Sinon, admettons qu'on ait des coefficients de Bézout $x, y$ pour $b, r$, tels que $$x b + y r = {\rm PGCD}(b, r) = {\rm PGCD}(a, b) .$$ Un simple calcul donne $u = y$ et $v = x - y q$.

*Remarque :* Ces coefficients ne sont pas uniques ! Pour s'en rendre compte, on peut simplement voir que la sortie si $r = 0$ pourrait être $u = \kappa$, $v = 1 - \kappa$ avec $\kappa$ quelconque. On peut aussi effectuer une modification $(u, v) \leftarrow (u - b, v + a)$ ou $(u, v) \leftarrow (u + b, v - a)$ sur la sortie sans modifier le résultat.

### 1.1. Coefficients de Bézout

**TODO:**  
Implémenter cet algorithme récursif dans la fonction `bezout` ci-dessous.

In [2]:
def bezout(a, b):
    """
    Donne deux entiers u et v tels que a * u + b * v = pgcd(a, b).

    Args:
        a (int): Un premier entier.
        b (int): Le second entier.

    Returns:
        int, int: Les entiers u et v tels que a * u + b * v = pgcd(a, b)
    """
    pass  # TODO

**TODO:**  
Tester la fonction ci-dessus, grâce à des calculs à la main des coefficients de Bézout des couples
- 46, 21
- 214, 76
- 417, 55

### 1.2. Calcul d'inverse

Si ${\rm PGCD}(a, b) = 1$, on peut trouver $a^{-1}$ dans $\mathbb{Z} / b\mathbb{Z}$. En effet, 
$$ ua + vb = 1\ \Rightarrow\ ua = 1\ [b]\ \Leftrightarrow\ a^{-1} = u\ [b] . $$
Réciproquement, si on trouve $u \in \mathbb{Z}$ tel que $ua = 1\ [b]$, alors ${\rm PGCD}(a, b) = 1$.

**TODO:**  
Coder une fonction `inv_mod(a, p)` qui calcule l'inverse de `a` modulo `p` lorsque cela est possible. Si l'inverse n'existe pas, la fonction devra renvoyer une erreur.

In [4]:
def inv_mod(a, p):
    """
    Calcule l'inverse de a modulo p. Si l'inverse n'existe pas,
    renvoyer une erreur.

    Args:
        a (int): Le nombre à inverser
        p (int): Le modulo

    Returns:
        int: L'inverse de a dans Z / pZ
    """
    pass  # TODO

**TODO:**  
Tester la fonction sur des exemples. Vérifier que cela ne fonctionne pas sur un exemple ${\rm PGCD}(a, b) \neq 1$.

### 1.3. Résolution d'équations

Ici, on cherche à résoudre l'équation $ax = b\ [n]$ dans le cas où la solution est unique, i.e. dans le cas ${\rm PGCD}(a, n) = 1$.

*Contre-exemple :* Si ${\rm PGCD}(a, n) \neq 1$, la solution peut ne pas être unique ou ne pas exister. Par exemple $3x = 0\ [6]$ admet pour solutions $0$ et $2$, et $3x = 1\ [6]$ n'admet pas de solution.

**TODO:**  
Coder une fonction `solve_mod` qui, à partir des données $(a, b, n)$ du problème ci-dessus, renvoie la solution $x$ de ce système lorsque cette dernière existe et est unique.

In [6]:
def solve_mod(a, b, n):
    """
    Résout l'équation a * x = b dans Z / nZ.

    Args: 
        a (int): Le nombre facteur de l'inconnue x
        b (int): Le nombre à droite de l'équation
        n (int): Le modulo

    Returns:
        int: La solution x de l'équation
    """
    pass  # TODO

**TODO:**  
1. Vérifier `solve_mod(a, 1, n) == inv_mod(a, n)` sur quelques exemples.
2. Vérifier `solve_mod(a, 0, n) == 0` sur quelques exemples.
3. Calculer la solution de $11 x = 6\ [14]$ à la main, et comparer au résultat de la fonction implémentée.

In [7]:
# 1.
# TODO

# 2.
# TODO

# 3.
# TODO

## Partie 2 - Manipulations dans $\mathbb{Z} / pq\mathbb{Z}$

Nous allons affiner nos connaissances sur l'objet $\mathbb{Z} / pq\mathbb{Z}$, notamment à travers un énoncé connu sous le nom de [«théorème des restes chinois»](https://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_des_restes_chinois). 
Nous ne l'énoncerons pas dans sa version la plus complète puisque seule une version *faible* nous suffira.

**Théorème faible des restes chinois:** Soit $p,q$ deux entiers premiers entre eux et notons $n = pq$. Alors:
$$ \mathbb{Z}/n\mathbb{Z} \simeq  \mathbb{Z}/p\mathbb{Z}\times\mathbb{Z}/q\mathbb{Z}.$$
Autrement dit, la correspondance $x \mod pq \mapsto (x \mod p, x \mod q)$ est inversible, et il existe une correspondance $(y_p \mod p, y_q \mod q) \mapsto y \mod pq$.


### 2.1. Décomposition dans l'espace produit

En pratique, tout élément de $\mathbb{Z}/n\mathbb{Z}$ correspond uniquement à un élément de $\mathbb{Z}/p\mathbb{Z}\times \mathbb{Z}/q\mathbb{Z}$. Plus précisément:
$$
x\mod n \rightarrow \begin{cases}
x\mod p, \\
x\mod q.
\end{cases}
$$

Par exemple, $14=7\times 2$ et:
$$
10\mod 14 \leftrightarrow \begin{cases}
0\mod 2, \\
3\mod 7.
\end{cases} 
$$
Nous allons coder les correspondances entre ces deux objets.
Dans la suite, on supposera alors toujours que $n=pq$ avec $p$ et $q$ premiers entre eux.

**TODO:**  
Coder une fonction `chinois_aller` qui associe à un élément modulo $n$ le couple de classe modulo $p$ et $q$. Tester votre fonction.

In [ ]:
def chinois_aller(x, p, q):
    """
    Associe à un entier sa classe de congruence modulo p et q

    Args: l'entier et les deux modulo voulu

    Outputs: un couple d'entier modulo p et q
    """
    pass  # TODO


# TODO test

### 2.2. Recomposition depuis l'espace produit

Maintenant, nous souhaitons coder la fonction qui inverse chinois_aller. Considérons $n=pq$ avec $p$ et $q$ premiers entre eux. Pour cela nous devons trouver l'unique élément $x$ de $\mathbb{Z}/n\mathbb{Z}$ résolvant le système d'équation suivant:
$$
\begin{cases} x = y_p\ [p], \\ x = y_q\ [q], \end{cases}
$$
où $y_p$ et $y_q$ sont donnés.

Pour ce faire, nous devons procéder en plusieurs étapes, que nous allons programmer au fur et à mesure.

#### 2.2.a. Les systèmes fondamentaux

On résout préalablement les deux systèmes suivants d'inconnues respectives $x_p$ et $x_q$ dans $\mathbb{Z}/n\mathbb{Z}$:
$$ \begin{cases} x_p = 1\ [p], \\ x_p = 0\ [q], \end{cases} \qquad\qquad \begin{cases} x_q = 0\ [p], \\ x_q = 1\ [q]. \end{cases} $$

Pour cela, il faut trouver une combinaison de Bézout entre $p$ et $q$, $up + vq = 1$.

*Remarque :* Ces solutions $x_p$ et $x_q$ ne dépendent que de $p$ et de $q$, pas de $y_p, y_q$. Si on souhaitait résoudre plusieurs systèmes $x = y_p\ [p]$, $x = y_q\ [q]$, on pourrait calculer ces solutions fondamentales $x_p, x_q$ une seule fois.

**TODO:**  
Justifier que $x_p = v q$ et $x_q = u p$ sont les solutions des deux systèmes. Puis, coder une fonction resol_inter qui à partir de $p$ et $q$ résout ces systèmes.

In [9]:
def resol_inter(p, q):
    """
    Étant donné les entiers p et q résout les deux systèmes ci-dessus

    Args: les moduli du système de congruences

    Outputs: une solution à chacun des deux systèmes dans l'ordre x_p et x_q.
    """
    pass  # TODO

#### 2.2.b. La résolution totale

Enfin, à partir de ces deux solutions $x_p$ et $x_q$ on peut construire une solution du système:
$$
\begin{cases} x = y_p\ [p], \\ x = y_q\ [q]. \end{cases}
$$ 

**TODO:**  
Justifier que $x = y_p x_p + y_q x_q$ est une solution du système ci-dessus. Puis, implémenter une fonction `chinois_retour` qui renvoie l'unique solution $x$ modulo $n$.

In [11]:
def chinois_retour(yp, yq, p, q):
    """
    Chercher la solution du système ci-dessus.

    Args: les quatres paramètres du système ci-dessus p,q,y_p,y_q

    Outputs: la solution modulo pq du système
    """
    pass  # TODO

**TODO**  
Vérifier expérimentalement que `chinois_retour` est bien l'inverse de la fonction `chinois_aller`.

## Partie 3 - Racines en arithmétique modulaire

Dans la partie 3.2 à arriver, nous avons implémenté [l'algorithme de Tonelli-Shanks](https://en.wikipedia.org/wiki/Tonelli%E2%80%93Shanks_algorithm) pour calculer une racine carrée d'un nombre dans $\mathbb{Z} / p\mathbb{Z}$.

### 3.1. La $p$-valuation

Dans l'algorithme 
Déjà codé par nos soins mais on a besoin de la $p$-valuation. Soit $p$ un entier, la $p$-valuation d'un entier $n$ est le plus grand exposant $s$ tel que:
$$
p^s \text{ divise } n.
$$

Par exemple, la $2$-valuation de $52$ est $2$ tandis que sa $13$-valuation est $1$ et sa $3$-valuation est $0$.

**TODO:**  
Coder la fonction `valuation` décrite ci-dessus et la tester.

In [ ]:
def valuation(x, p):
    """
    Cherche l'exposant maximal s tel que x = p^s * r
    
    Args: le deuxième argument indique quelle p-valuation doit être calculé pour le premier argument.
    
    Output: le plus grand exposant s tel que p^s divise n et le quotient de la division de x par p^s.
    """
    pass  # TODO


# TODO test

### 3.2. Algorithme de Tonelli-Shanks

Le code suivant est une méthode implémentée par nos soins pour trouver une racine carrée de $x$ modulo $p$.
Vous n'avez pas besoin de comprendre cet algorithme.

Nous utilisons la bibliothèque `galois`, très pratique pour tout ce qui touche à l'arithmétique modulaire, qu'il s'agisse de calculer dans les anneaux de congruences ou de manipuler des nombres premiers.
Une brève introduction à ce module est disponible en Partie 5 «Pour aller plus loin», et nous l'utiliserons pour le benchmark en Partie 4.

**TODO**  
Vérifiez que cette fonction renvoie bien une racine sur un exemple de votre choix.

In [ ]:
def sqrt_mod(x, p):
    """
    Algorithme de Tonelli-Shanks pour chercher une racine carrée
    de `x` modulo `p` avec `p` premier.

    Grosse utilisation du petit théorème de Fermat: a ** (p-1) = 1.
    Notamment, on vérifie que $x^{(p - 1) / 2} = y^{p - 1} = 1$ avant
    de procéder à la recherche d'un carré.

    Args:
        x (int):
    """
    if p == 2:
        return x

    gf = galois.GF(p)
    x, unit = gf(x), gf(1)

    pm1, pp1 = p - 1, p + 1
    n = pm1 // 2

    # Si x n'est pas un carré, renvoyer zéro
    if x**n != unit:
        return 0

    # Si $p = 4k + 3$, on pose $y = x^{k+1} = x^{(p+1) / 4}$, car dans
    # ce cas, $y^2 = x^{(p+1)/2} = x^{(p-1)/2} \times x = 1 \cdot x$
    # grâce au petit théorème de Fermat.
    if pp1 % 4 == 0:
        return (x ** (pp1 // 4)).item()

    # Cas $p = 4k + 1$, basée sur la décomposition Z/pm1 Z = Z/2^s Z \times Z/tZ

    # Extraction de la 2-valuation de p - 1
    s, t = valuation(pm1, 2)  # s ≥ 2 car p - 1 = 4k

    # Recherche d'un nombre qui n'est pas un carré
    u = gf(2)
    while u**n == unit:  # <=> u ** n != -1
        u += unit

    v = u**t  # racine primitive 2^s - ième de l'unité
    z = x ** ((t + 1) // 2)  # z^2 = x * x^t => on cherche x^-t

    xt, inv_v = x**t, unit / v
    inv_v2 = inv_v**2

    # l = 0
    inv_vl, inv_v2l = unit, unit  # == v ** (-l), v ** (2 * l)
    for i in range(s - 2):
        S = 2 ** (s - 2 - i)
        if (xt * inv_v2l) ** S != unit:
            # l += 2 ** i
            inv_vl *= inv_v ** (2**i)
            inv_v2l *= inv_v2 ** (2 ** (i + 1))

    sqrt_x = z * inv_vl
    return sqrt_x.item()  # conversion gf -> int


sqrt_mod(18, 41) ** 2 % 41

### 3.3. Chiffrement (numérique) de Rabin faible

La procédure de chiffrement de Rabin est directe : il suffit de calculer $c = m^2 \mod n$.
Pour déchiffrer $c$, en revanche, il y a trois étapes :
1. décomposition $c \in \mathbb{Z} / n\mathbb{Z} \mapsto (a, b) \in \mathbb{Z}/p\mathbb{Z} \times \mathbb{Z}/q\mathbb{Z}$
2. racines $\sqrt{a}, \sqrt{b}$ et leurs opposées.
3. résolution des systèmes pour obtenir les 4 valeurs possibles de $m$,
    - $m = \sqrt{a}\ [p], m = \sqrt{b}\ [q]$ ;
    - $m = \sqrt{a}\ [p], m = -\sqrt{b}\ [q]$ ;
    - $m = -\sqrt{a}\ [p], m = \sqrt{b}\ [q]$ ;
    - $m = -\sqrt{a}\ [p], m = -\sqrt{b}\ [q]$.
4. faire correspondre ces quatre éventualités à un message dans $\mathbb{Z}/n\mathbb{Z}$.

Ici, on ignore la partie 4 qui demanderait une correspondance nombre-caractère.

**TODO:**  
Écrire une fonction `Rabin_encrypt` qui chiffre le message d'Alice, et une fonction `Rabin_decrypt` qui donne les quatres messages en clairs possibles.

Pour les tests pratiques on pourra considérer $p=7757$ et $q=7759$. Ainsi $n=7757\times 7759$.

In [15]:
def Rabin_encrypt(n, m):
    """
    Chiffre le message m par la procédure de Rabin

    Args: la clé publique pq et le message à chiffrer sous forme d'entier

    Outputs: Le message chiffré modulo n d'Alice
    """
    pass  # TODO

In [16]:
def Rabin_decrypt(c, p, q):
    """
    Fonction déchiffrant le message chiffré par la procédure de Rabin

    Args: le message chiffré et les deux clefs privéses

    Outputs: les quatres messages en clair éventuels
    """
    pass  # TODO

Pour des raisons de temps nous n'explorerons pas plus en profondeur cette piste. Un retour de cette méthode sera envisageable plus tard... 
En pratique, des test suplémentaires sont ensuite effectués sur les sorties pour déterminer quel est le message en clair d'origine.

## Partie 4 - Décomposition en facteurs premiers

Une grande partie des algorithmes de cryptographie repose sur le fait que trouvé la décomposition en produits de facteurs premiers est une tâche longue comparée aux autres. Nous allons ici présenté deux algorithmes de factorisation.

De meilleurs algorithmes existent de nos jours, mais ces derniers restent encore long. Par exemple, la méthode de Lehman, n'a une complexité que de $\mathcal{O}\bigl(n^{\frac{1}{3}}\bigr)$, par rapport à $\mathcal{O}(\sqrt{n})$ pour la recherche exhaustive de Fermat.

### 4.1. Recherche exhaustive de diviseurs

Pour trouver un diviseur d'un entier, on peut se contenter de vérifier si ce dernier est est divisible par tous les entiers plus petit que sa racine carrée, tous les entiers $k \leq \lfloor \sqrt{n} \rfloor$.

**TODO**  
Coder une fonction `fact_exhaustive` qui cherche le plus petit diviseur $\neq 1$ d'un entier $n$ donné en entrée.

In [ ]:
def fact_exhaustive(n):
    """
    Teste les les entiers inférieurs à n, un à un.

    Args: un entier impair à factoriser

    Returns: le plus petit diviseur de n
    """
    pass  # TODO


# TODO test

### 4.2. Racine de grands entiers

Comment calculer cette valeur maximale $\lfloor \sqrt{n} \rfloor$ ?
Une traduction naturelle de cette formule en Python serait d'utiliser le module `math`, avec `sqrt_n = math.sqrt(n)` pour calculer la racine et `math.floor(sqrt_n)` pour en prendre la partie entière.
Le souci est que cette méthode calcule la racine avec des nombre flottants et peut avoir un mauvais comportement pour les très grands entiers.
Pour cette raison, nous privilégerons la fonction `galois.isqrt` qui ne présente pas ces soucis.

Un bon moyen d'observer cette limite est de coder une fonction qui vérifie si un nombre est un carré.
Il y a au moins trois implémentations possibles à cette fonction :
1. Calculer `math.sqrt(n)` et vérifier s'il s'agit d'un entier avec la méthode `.is_integer()`
2. Calculer la partie entière de la racine carrée de $n$ et vérifier que son carré donne bien $n$,
    - soit avec le module `math`
    - soit avec le module `galois`

*Remarque :* Cette dernière approche est plus lente ! Mais c'est la seule qui n'implique jamais de nombre à virgule flottante.

**TODO:**  
Écrire toutes ces implémentations.
Comparer leurs résultats avec $n = (10^{18} + 1)^2$ et $n = 10^{22} + 1$.
Comment pourriez-vous modifier votre implémentation de la fonction `fact_exhaustive` précédente ?

In [1]:
# TODO implémentations et test comparatif

### 4.3. Étude de complexité

On cherche à étudier la complexité de notre algorithme de factorisation pour les entiers de la forme $n = p q$ avec $p$ et $q$ premiers.
Le jeu de données utilisé pour l'analyse doit être choisi précautionneusement : il y a une foultitude de valeurs possibles pour $p$ et $q$. 
Est-ce que leurs valeurs doivent être indépendantes ? 
Est-ce qu'elles doivent être du même ordre de grandeur ?

Ici, on prend le parti de générer les nombres premiers «par blocs» en fixant un nombre de bits et en tirant aléatoirement plusieurs nombres premiers différents qui ont ce nombre de bits.
Ainsi, on s'assure une distribution quasi-uniforme des nombres en échelle logarithmique, et en les choisissant tous différents, on s'assure que la liste des nombres premiers de `galois` dans `galois.factors` ne sera pas utilisée ou presque.
Cela correspond au contexte utilisé en cryptographie : lorsqu'on génère les clés privées, on souhaite que les nombres premiers $p$ et $q$ soient **tous les deux grands**.

**TODO**  
Évaluer la complexité de la factorisation en fonction de $n$ en utilisant le module `time` et la tracer.

In [ ]:
def generate_primes(
    b_min=10, b_max=20, b_step=1, n_per_bit=10, seed=DEFAULT_SEED
):
    """
    Génère des produits n = p * q avec des nombres premiers p, q
    vérifiant 2^b_min ≤ p, q ≤ 2^b_max. Pour tout b entre b_min et
    b_max avec un pas b_step, on tire aléatoirement n_per_bit nombres
    premiers *uniques* p, q tels que 2^b ≤ p, q < 2^(b+1).
    """
    bits = range(b_min, b_max + 1, b_step)
    all_n = [0] * (len(bits) * n_per_bit)
    all_p, all_q = all_n.copy(), all_n.copy()

    rng = random.Random(seed)
    for ib, b in enumerate(bits):
        primes_b = set()  # set of primes generated with b bits
        for i in range(n_per_bit):
            p, q = 0, 0
            while p in primes_b or q in primes_b or p == q:
                seed_p, seed_q = rng.getrandbits(64), rng.getrandbits(64)
                p = galois.random_prime(b, seed=seed_p)
                q = galois.random_prime(b, seed=seed_q)

            primes_b.add(p)
            primes_b.add(q)
            idx = ib * n_per_bit + i
            all_p[idx], all_q[idx], all_n[idx] = p, q, p * q

    return all_p, all_q, all_n


bench_p, bench_q, bench_n = generate_primes(b_max=20, n_per_bit=10)
# Sanity checks: nombre d'entiers générés, unicité et disjonction de bench_p, bench_q
len(bench_p), len(set(bench_p).union(bench_q)) - len(bench_p) - len(bench_q)

## Partie 5 - Pour aller plus loin

Grâce à la bibliothèque matplotlib, tracer les temps d'éxécution des deux algorithmes de factorisation en fonction du nombre de bits sur lequel l'entier auquel il est appliqué est codé. Mettre dans une échelle correcte pour identifier la complexité de ces algorithmes et observer les pires et meilleurs cas.

### 5.1. Détails sur `galois`

La fonction galois.GF(p) spécifie que nous travaillons modulo p avec p premier et nous autorise à faire les opérations algèbriques usuelles.

**Attention:** faites bien attention de mettre un nombre premier dans cette fonction au risque de voir des phénomènes bizarres.

In [ ]:
import galois
x = galois.GF(5)(2) / galois.GF(5)(2)
x.item()

In [ ]:
mod13 = galois.GF(13)
x = mod13(8) + mod13(9)
print(x)

### 5.2. L'algorithme de factorisation de Fermat

Cet algorithme repose sur l'idée suivante, si $n=rs$ avec $n$ impair. Alors:
$$
n = (a + b)(a - b) = a^2 - b^2
$$
L'algorithme de factorisation de Fermat propose alors de calculer:
1. vérifier que $n$ n'est pas pair,
2. chercher des valeurs $a$ pour que $a^2-n$ soit un carré (donc $a>\sqrt{n})$, 
3. en posant $b=\sqrt{a^2-n}$, on retourne $a+b$ et $a-b$. 

**TODO**  
Programmer une fonction `fact_Fermat` qui implémente cet algorithme. On pourra utiliser la fonction `is_square` des parties précédentes.

In [31]:
def fact_Fermat(n):
    """
    Recherche un diviseur par la méthode de Fermat pour un entier impair expliquée ci-dessus.

    Args:
     n (int): L'entier à factoriser

    Returns:
        int, int: Un couple d'entiers dont le produit donne l'entrée
    """
    pass  # TODO